# Final Capstone: Revisiting the Netflix Prize

## Notebook 6: Models

### A Note on Expectations

Very early in the Netflix Contest, a brilliant young computer scientist who goes by the alias 'Simon Funk' openly revealed information which brought him a great deal of notoriety (and many job offers). He revealed here: https://sifter.org/~simon/journal/20061211.html some groundbreaking information on how Singular Value Decomposition can be utilized to predict the missing ratings of a sparse User-Item matrix. This method is commonly regarded as the most powerful prediction tool for this type of data. Unfortunately, the research required to incorporate this method would consume a much greater amount of time than is allotted for this capstone. While expect to build models that improve on baseline results, I do not expect to produce an RMSE that would land atop the leaderboards.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from scipy import stats
from scipy.stats import yeojohnson as yj
from sklearn import metrics
from sklearn.linear_model import LinearRegression, SGDRegressor, Lasso
from sklearn.linear_model import Ridge, OrthogonalMatchingPursuit, Lars
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler

sns.set_style('darkgrid')
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.8f}'.format)

In [2]:
%%time
# import main training and quiz data
base_path = 'C:/Users/jnpol/Documents/DS/Data Science/UL/'
train_pca = pd.read_parquet(base_path + 'train_pca.parquet')
train_trans = pd.read_parquet(base_path + 'train_trans.parquet')
train_target = pd.read_parquet(base_path + 'train_target.parquet')
quiz_pca = pd.read_parquet(base_path + 'quiz_pca.parquet')
quiz_trans = pd.read_parquet(base_path + 'quiz_trans.parquet')
quiz_target = pd.read_parquet(base_path + 'quiz_target.parquet')

quiz_pca.info()
train_trans.info()
quiz_trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1408395 entries, 0 to 1408394
Data columns (total 17 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   mov_id           1408395 non-null  int16  
 1   cust_id          1408395 non-null  int32  
 2   day_rated        1408395 non-null  int16  
 3   mov_year         1408395 non-null  int16  
 4   avg_rate_pm_pd   1408395 non-null  float32
 5   avg_rate_pc_pd   1408395 non-null  float32
 6   cust_day_count   1408395 non-null  int16  
 7   cust_days_since  1408395 non-null  int16  
 8   mov_days_since   1408395 non-null  int16  
 9   mov_avg_rating   1408395 non-null  float32
 10  cust_avg_rating  1408395 non-null  float32
 11  mov_day_avg      1408395 non-null  float32
 12  cust_day_avg     1408395 non-null  float32
 13  avg_rate_yr      1408395 non-null  float32
 14  avg_rate_cst_yr  1408395 non-null  float32
 15  bline_approx     1408395 non-null  float32
 16  quiz_pc          1

### Time Matters

Time is crucial in all walks of life. With such a massive volume of data, it is quite reasonable to model on fractions of the dataset instead of the entire 100,000,000+ observations. I have done model runs across the spectrum of fractions, and there is very little difference (usually at the third decimal place). Perhaps if the prize was still up for grabs, I may reconsider this!

In [3]:
%%time
train_pca = train_pca.sample(frac=0.8, random_state=11)
train_trans = train_trans.sample(frac=0.8, random_state=11)
train_target = train_target.sample(frac=0.8, random_state=11)

Wall time: 1min 6s


In [5]:
%%time
train_pca = train_pca[['cust_id', 'avg_rate_pm_pd', 'mov_avg_rating', 'cust_avg_rating']]
quiz_pca = quiz_pca[['cust_id', 'avg_rate_pm_pd', 'mov_avg_rating', 'cust_avg_rating']]

Wall time: 646 ms


In [6]:
%%time
tmix = pd.concat([train_pca, train_trans.tyr_trans, train_trans.tary_3rt], axis=1)
qmix = pd.concat([quiz_pca, quiz_trans.qyr_trans, quiz_trans.qary_3rt], axis=1)

Wall time: 817 ms


In [7]:
%%time
# training set
scaler = StandardScaler()
X_train = scaler.fit_transform(tmix)
y_train = train_target.rating.to_numpy()

# quiz set
scaler = StandardScaler()
X_test = scaler.fit_transform(qmix)
y_test = quiz_target.rating.to_numpy()

Wall time: 10.9 s


In [11]:
# algos
rid = Ridge(max_iter=10, random_state=413)
sgdr = SGDRegressor(alpha=5, max_iter=10, shuffle=False, verbose=1, random_state=761)
hgbr = HistGradientBoostingRegressor(learning_rate=0.2, max_iter=250, max_leaf_nodes=200, random_state=119)

## Ridge Regression

Ridge Regression was the top performer among the generalized linear models in terms of both score and computational efficiency. Utilized but discarded due to inferior results were: LinearRegression, OrthogonalMatchingPursuit, Lars, HuberRegressor, and TweedieRegressor.

In [9]:
%%time
ridg = rid.fit(X_train, y_train)
y_pred = ridg.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print('R-squared =', ridg.score(X_test, y_test))
print('RMSE =', rmse)

R-squared = 0.2234926118452003
RMSE = 0.993489507504251
Wall time: 4.35 s


## Stochastic Gradient Descent

The results from this model were difficult to make sense of.  When certain combinations of features were selected, the Ridge results would improve, and this SGDRegressor would produce worse scores.  Other combinations would produce the opposite result.  Ultimately, a StackingRegressor might produce a blended result that is superior to these algorithms taken individually.

In [10]:
sgd = sgdr.fit(X_train, y_train)
y_pred = sgd.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print('R-squared =', sgd.score(X_test, y_test))
print('RMSE =', rmse)

-- Epoch 1
Norm: 0.10, NNZs: 6, Bias: 3.606274, T: 79257690, Avg. loss: 0.533659
Total training time: 5.05 seconds.
-- Epoch 2
Norm: 0.10, NNZs: 6, Bias: 3.604844, T: 158515380, Avg. loss: 0.533619
Total training time: 10.12 seconds.
-- Epoch 3
Norm: 0.10, NNZs: 6, Bias: 3.604104, T: 237773070, Avg. loss: 0.533612
Total training time: 15.09 seconds.
-- Epoch 4
Norm: 0.10, NNZs: 6, Bias: 3.603627, T: 317030760, Avg. loss: 0.533607
Total training time: 20.05 seconds.
-- Epoch 5
Norm: 0.10, NNZs: 6, Bias: 3.603286, T: 396288450, Avg. loss: 0.533605
Total training time: 25.01 seconds.
-- Epoch 6
Norm: 0.10, NNZs: 6, Bias: 3.603026, T: 475546140, Avg. loss: 0.533602
Total training time: 29.96 seconds.
Convergence after 6 epochs took 29.96 seconds
R-squared = 0.07453113413308265
RMSE = 1.084604483013789


## HistogramGradientBoostingRegressor

This model was quite a discovery. It is available via sklearn, but declared as being 'experimental'. The algorithm is incredibly fast compared with the standard GradientBoostingRegressor, and the hyperparameters are quite intuitive.

In [12]:
%%time
hgb = hgbr.fit(X_train, y_train)
y_pred = hgb.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print('R-squared =', hgb.score(X_test, y_test))
print('RMSE =', rmse)

R-squared = 0.22423733620616892
RMSE = 0.9930129806011876
Wall time: 16min 42s


## Conclusions

In terms of a best model, the HistogramGradientBoostingRegressor produced the best scores, while most of the GLM's were just slightly behind.  KNN Regression was promising in early tests, but the algorithm simply took too long to train and fit.

In my estimation there are two major takeaways from this research that I hope will provide value to any data science practitioner. The first, is that datasets in general should be evaluated for the possibility of engineering new features. One should seek insights and relationships in the data that may not be obvious at first glance. This research showed a great improvement in correlation coefficients with the engineered features. I believe this approach can, should, and will be applied more frequently in the industry. Finally, the big data aspect of this project was quite revealing. Working with this dataset showed limitations both in my hardware, and even in Dask, which ironically designed for big data. I could not utilize Dask's client and workers on my machine - even using 10GB allocations for the workers. Once I needed to compute something, a warning would display, showing that 95% of the allocated memory for the workers had been exceeded, and the operation would terminate. While this was unfortunate, there were benefits from this problem. I was forced to consider every operation in terms of computational efficiency. There are several ways to process lists, series, arrays, rows, and columns. I quickly learned that Numpy set operations were extremely more powerful to conditionally select data in lists or arrays than using for or while loops. Additionally, iterating rows of a DataFrame and performing conditional operations is a dreadfully slow process. The point here, is that each operation has an associated computational cost. With small to medium datasets, we may not notice the difference. But with big data, you can be certain to encounter these challenges at several stages of the workflow, if not constantly present.